En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

Se detallará el cómo se trabajó el archivo .json para cada una de las funciones. Tambien se considerarán todas las posibles mejoras necesarias al momento de trabajar los datos. El cómo trabajar el archivo debería ser similar para cada caso, con algunos ejercicios requiriendo mas proceso que otros.

Para el primer ejercicio se deberia priorizar un count por fecha en tweets, luego de cada fecha con count mas alto, hacer el count de los usuarios con mas publicaciones. Como solo se necesita la fecha y el autor, probablemente se pueda trabajar el archivo para reducir carga en busqueda por iteración.

Similar al anterior ejercicio, count por emoji que se encuentre, ordenar y retornar los 10 mas altos. En este caso solo se requiere el contenido del tweet y no los demas parametros. Transformar para el retorno el unicode en emoji.

Nuevamente, contar los @usuarios que existan por post, ordenar y retornar los mas altos.

Inicialmente para las soluciones mas rapidas, usaremos principalmente pandas para su manejo eficiente de columnas, dado que hay harto grouping que hacer.

In [125]:
import json 
import pandas as pd


In [126]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [127]:
%%memit
file_path = "../../farmers-protest-tweets-2021-2-4.json"
data=[]
with open(file_path, 'r') as file:
    for line in file:
        obj = json.loads(line)
        data.append(obj)
        
df = pd.DataFrame(data)


peak memory: 2972.70 MiB, increment: -181.41 MiB


Se agregan columnas a conveniencia, editando "date" para trabajarse con año-mes-dia
Además se agrega la columna username, que extrae el "username" del objeto "user" presente en cada fila.
Se generan los dataframes, uno con los dias de mas tweets y otro con los usuarios que mas postearon en cada día, esto luego se mergea para tener el resultado final.

In [128]:
%%memit
df['date'] = pd.to_datetime(df['date'])
df['date_only'] = df['date'].dt.date
df['username'] = df['user'].apply(lambda x: x['username'])

grouped = df.groupby(['date_only', 'username']).size().reset_index(name='count')
most_frequent_user = grouped.loc[grouped.groupby('date_only')['count'].idxmax()]
print("\nMost frequent username for each date:")
print(most_frequent_user)

grouped_df=df.groupby('date_only').size().reset_index(name='count')

print(len(most_frequent_user))
print(len(grouped_df))





Most frequent username for each date:
        date_only         username  count
2740   2021-02-12  RanbirS00614606    176
7536   2021-02-13  MaanDee08215437    178
15136  2021-02-14    rebelpacifist    119
18540  2021-02-15           jot__b    134
23260  2021-02-16           jot__b    133
26577  2021-02-17   RaaJVinderkaur    185
33193  2021-02-18  neetuanjle_nitu    195
35219  2021-02-19         Preetm91    267
38382  2021-02-20  MangalJ23056160    108
42691  2021-02-21       Surrypuria    161
46533  2021-02-22   preetysaini321    110
48696  2021-02-23       Surrypuria    135
51458  2021-02-24   preetysaini321    107
13
13
peak memory: 2905.12 MiB, increment: -304.83 MiB


In [129]:
%%memit

join=pd.merge(most_frequent_user,grouped_df, on='date_only')
sorted_join = join.sort_values(by='count_y', ascending=False)

print(sorted_join)

     date_only         username  count_x  count_y
0   2021-02-12  RanbirS00614606      176    12347
1   2021-02-13  MaanDee08215437      178    11296
5   2021-02-17   RaaJVinderkaur      185    11087
4   2021-02-16           jot__b      133    10443
2   2021-02-14    rebelpacifist      119    10249
6   2021-02-18  neetuanjle_nitu      195     9625
3   2021-02-15           jot__b      134     9197
8   2021-02-20  MangalJ23056160      108     8502
11  2021-02-23       Surrypuria      135     8417
7   2021-02-19         Preetm91      267     8204
9   2021-02-21       Surrypuria      161     7532
10  2021-02-22   preetysaini321      110     7071
12  2021-02-24   preetysaini321      107     3437
peak memory: 3037.06 MiB, increment: -68.02 MiB


Ahora con este dataframe solo falta tomar los 10 primeros, pasarlos a una lista de tuplas para el retorno de q1

In [130]:
%%memit

dateuser=sorted_join[['date_only','username']].head(10)
q1_t = [tuple(x) for x in dateuser.values]
print(q1_t)


[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
peak memory: 3128.62 MiB, increment: -213.75 MiB


Q2 Es un problema mucho mas simple, utilizando la libreria emoji y counter se resuelve rapidamente, siendo eficiente, dentro de lo posible, en el uso de memoria y tiempo.


In [131]:
%%memit

import emoji
from collections import Counter

with open(file_path, 'r') as file:
    emoji_counts = Counter()
    for line in file:
        obj = json.loads(line)
        for char in obj["content"]:
            if char in emoji.EMOJI_DATA:
                emoji_counts[char] += 1

emoji_counts=sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]

print(emoji_counts)



    


[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]
peak memory: 2815.64 MiB, increment: -123.66 MiB


Q3 Es similar a Q2, con la diferencia que esta vez se utilizan expresiones regulares para detectar las menciones en usuarios en el contenido de cada tweet.

In [132]:
%%memit

import re

with open(file_path, 'r') as file:
    dif_mentions = Counter()
    mention = re.compile(r'@(\w+)')

    for line in file:
        obj = json.loads(line)
        mentions = mention.findall(obj["content"])
        dif_mentions.update(mentions)
print (dif_mentions.most_common(10))



[('narendramodi', 2261), ('Kisanektamorcha', 1836), ('RakeshTikaitBKU', 1639), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1015), ('rihanna', 972), ('UNHumanRights', 962), ('meenaharris', 925)]
peak memory: 2840.08 MiB, increment: -246.41 MiB


Tanto Q2 como Q3 son problemas simples que no tienen variación en caso de generar soluciones distintas para mejor tiempo o mejor memoria. Por otra parte Q1 si es trabajable, dado que todo el grouping y el uso de dataframes puede ser reducido a costa de hacer mas iteraciones pero ocupando menos memoria.

Se hara una implementacion de Q1 sin pandas, buscando realizar las operaciones lo mas inplace posible.

In [140]:
%%memit

file_path = "../../farmers-protest-tweets-2021-2-4.json"
from datetime import datetime, date

with open(file_path, 'r') as file:
    date_count=Counter()
    for line in file:
        obj = json.loads(line)
        dt = datetime.fromisoformat(obj['date'])
        date = dt.strftime('%Y-%m-%d')
        date_count[date]+=1
    top_dates=date_count.most_common(10)

print(top_dates)

        


[('2021-02-12', 12347), ('2021-02-13', 11296), ('2021-02-17', 11087), ('2021-02-16', 10443), ('2021-02-14', 10249), ('2021-02-18', 9625), ('2021-02-15', 9197), ('2021-02-20', 8502), ('2021-02-23', 8417), ('2021-02-19', 8204)]
peak memory: 2298.62 MiB, increment: -152.70 MiB


In [134]:
%%memit

date_user=[]
for dates in top_dates:
        user_count=Counter()
        with open(file_path, 'r') as file:
            for line in file:
                obj = json.loads(line)
                dt = datetime.fromisoformat(obj['date'])
                date = dt.strftime('%Y-%m-%d')
                if date==dates[0]:
                    user = obj['user']['username']
                    user_count[user]+=1
            dt = datetime.strptime(dates[0], '%Y-%m-%d')
            date_obj = dt.date()
            date_user.append((date_obj,user_count.most_common(1)[0][0]))
print(date_user)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
peak memory: 1809.86 MiB, increment: -297.02 MiB


Vemos que el PEAK de memoria utilizada fueron 2298.62 MiB mientras que el codigo utilizando pandas ocupó 3128.62 MiB.

In [141]:
print("Mejora de espacio: ", ((3128/2298)-1)*100, "% menos espacio")
print("Empeoramiento en tiempo: ", (((23.6+2.9)/(10.4+0.4+1.1+0.4))-1)*100, "% mas lento")

Mejora de espacio:  36.118363794604 % menos espacio
Empeoramiento en tiempo:  115.4471544715447 % mas lento


Vemos que se puede obtener mejor uso de memoria pero a un alto costo en tiempo, dado que pandas vectoriza y realiza operaciones en columnas de forma mucho mas optima.

Finalmente, en caso de trabajar con archivos considerablemente mas grandes, seria oportuno ocupar soluciones cloud como Dataflow, por ejemplo para Q1 *personalmente* utilizaría la solución con pandas como script.